# BERT Meets Cranfield - Regression Approach
instead of classifing the lot we can also do regression. The trick is to assign a good conversion table in the first place.

## Environement Setup
Connect to Drive by clicking on the icon, this only works when this colab is generated via the Drive menu

In [1]:
%cd /content/drive/MyDrive/COMPUTING SCIENCE/THESIS_PROJECT/bert-meets-cranfield-regression/Code
# %cd /home/jupyter/BERT-BM25-Thesis-Project/bert-meets-cranfield-multilabel/Code

/content/drive/MyDrive/COMPUTING SCIENCE/THESIS_PROJECT/bert-meets-cranfield-regression/Code


In [2]:
!pip3 install -r ../requirements.txt

In [3]:
import utils
import data_utils
from operator import itemgetter
import os
import numpy as np

import torch
import importlib
import timeit
# from transformers import BertForSequenceClassification, BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction


In [4]:
# call after making any changes in utils.py
importlib.reload(utils) 
importlib.reload(data_utils) 

<module 'data_utils' from '/content/drive/MyDrive/COMPUTING SCIENCE/THESIS_PROJECT/bert-meets-cranfield-regression/Code/data_utils.py'>

## Notes
The following changes apply:
 * labels have been replaced with the regression_labels
 * the prepare model routine now works with a `num_labels=1` setting the taks head in regression mode

## Experiments
```python
reference conversion 0=0 and the rest is 1     #                EXPERIMENT A) overal NDCG 0.5389 
conversion_table = [[0, 1, 2, 3, 4],[0.1, 1, 0.9, 0.8, 0.6, 0.5]]
conversion_table = [[0, 1, 2, 3, 4],[1, 10, 8, 6, 5]] #         EXPERIMENT B) overal NDCG 0.5335 Not to be pusued
conversion_table = [[0, 1, 2, 3, 4],[0, 50, 45, 40, 35]] #      EXPERIMENT C) overal NDCG 0.5142 Not to be pusued
conversion_table = [[0, 1, 2, 3, 4],[-0.5, 1, 0.8, 0.6, 0.3]] # EXPERIMENT D) overal NDCG 0.5424
conversion_table = [[0, 1, 2, 3, 4],[-1, 1, 0.9, 0.8, 0.1]] #     EXPERIMENT E) overal NDCG 0.5388
```
move on to a log approach, if the score $s$ ranges [1,5], then $1-log(s)$ might be interesting, since they will resemble the best experiment:
```python
conversion_table = [[0, 1, 2, 3, 4],[-1, 1, 0.7, 0.53, 0.4]] # EXPERIMENT G) overal NDCG 0.5424
```


In [5]:
conversion_table = [[0, 1, 2, 3, 4],[-1, 1, 0.7, 0.53, 0.4]] #     EXPERIMENT F) overal NDCG 

In [6]:
# import utils
# import data_utils
# from operator import itemgetter
# import os

# ========================================
#               Hyper-Parameters
# ========================================
SEED = 76
MODE = 'Re-ranker'
MODEL_TYPE = 'bert-base-uncased'
LEARNING_RATE = 2e-5
MAX_LENGTH = 128
BATCH_SIZE = 32
EPOCHS = 1
TOP_BM25 = 100
MAP_CUT = 100
NDCG_CUT = 20
if MODE == 'Full-ranker':
    TEST_BATCH_SIZE = 1400
else:
    TEST_BATCH_SIZE = 100

CONVERSION_TABLE = [[0, 1, 2, 3, 4],[0, 1, 1, 1, 1]] 

# Set the seed value all over the place to make this reproducible.
utils.initialize_random_generators(SEED)

# if __name__ == "__main__":
def train_test():
    print("# ========================================")
    print("#               Hyper-Parameters")
    print(MODE)
    print(MODEL_TYPE)
    print(LEARNING_RATE)
    print(MAX_LENGTH)
    print(BATCH_SIZE)
    print(EPOCHS)
    print("# ========================================")
    print("#               Experiment-Settings")
    print("CONVERSION TABLE:")
    print(CONVERSION_TABLE )
    print("# ========================================")

    start = timeit.default_timer()

    device = utils.get_gpu_device()
    if not os.path.exists('../Output_Folder'):
        os.makedirs('../Output_Folder')

    queries = data_utils.get_queries('../Data/cran/cran.qry')
    corpus = data_utils.get_corpus('../Data/cran/cran.all.1400')
    rel_fed = data_utils.get_judgments('../Data/cran/cranqrel')

    # labels = utils.get_binary_labels(rel_fed)

    # get labels for the regression input
    regression_labels = utils.get_regression_labels(rel_fed, CONVERSION_TABLE)

    tokenized_corpus = [doc.split(" ") for doc in corpus]
    tokenized_queries = [query.split(" ") for query in queries]

    bm25, bm25_top_n = utils.get_bm25_top_results(tokenized_corpus, tokenized_queries, TOP_BM25)

    padded_all, attention_mask_all, token_type_ids_all, temp_feedback = utils.bert_tokenizer(MODE, bm25_top_n, corpus,
                                                                                            #  labels, queries,
                                                                                             regression_labels, queries,
                                                                                             MAX_LENGTH, MODEL_TYPE)

    # ========================================
    #               Folds
    # ========================================
    mrr_bm25_list, map_bm25_list, ndcg_bm25_list = [], [], []
    mrr_bert_list, map_bert_list, ndcg_bert_list = [], [], []
    mrr_bm25, map_bm25, ndcg_bm25 = 0, 0, 0
    mrr_bert, map_bert, ndcg_bert = 0, 0, 0

    for fold_number in range(1, 6):
        print('======== Fold {:} / {:} ========'.format(fold_number, 5))
        train_index, test_index = data_utils.load_fold(fold_number)

        padded, attention_mask, token_type_ids = [], [], []
        if MODE == 'Re-ranker':
            padded, attention_mask, token_type_ids = padded_all, attention_mask_all, token_type_ids_all
        else:
            temp_feedback = []
            for query_num in range(0, len(bm25_top_n)):
                if query_num in test_index:
                    doc_nums = range(0, 1400)
                else:
                    doc_nums = bm25_top_n[query_num]
                padded.append(list(itemgetter(*doc_nums)(padded_all[query_num])))
                attention_mask.append(list(itemgetter(*doc_nums)(attention_mask_all[query_num])))
                token_type_ids.append(list(itemgetter(*doc_nums)(token_type_ids_all[query_num])))
                temp_feedback.append(list(itemgetter(*doc_nums)(regression_labels[query_num])))
                # temp_feedback.append(list(itemgetter(*doc_nums)(labels[query_num])))

        train_dataset = data_utils.get_tensor_dataset(train_index, padded, attention_mask, token_type_ids,
                                                      temp_feedback)
        test_dataset = data_utils.get_tensor_dataset(test_index, padded, attention_mask, token_type_ids, temp_feedback)

        mrr_bm25, map_bm25, ndcg_bm25, mrr_bm25_list, map_bm25_list, ndcg_bm25_list = utils.get_bm25_results(
            mrr_bm25_list, map_bm25_list, ndcg_bm25_list, test_index, tokenized_queries, bm25, mrr_bm25, map_bm25,
            ndcg_bm25, rel_fed, fold_number, MAP_CUT, NDCG_CUT)

        train_dataloader, test_dataloader, model, optimizer, scheduler = utils.model_preparation(MODEL_TYPE, train_dataset,
                                                                                                 test_dataset,
                                                                                                 BATCH_SIZE, TEST_BATCH_SIZE,
                                                                                                 LEARNING_RATE, EPOCHS)
        # ========================================
        #               Training Loop
        # ========================================
        epochs_train_loss, epochs_val_loss = [], []
        for epoch_i in range(0, EPOCHS):
            # ========================================
            #               Training
            # ========================================
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, EPOCHS))
            print('Training...')
            model, optimizer, scheduler = utils.training(model, train_dataloader, device, optimizer, scheduler)
        # ========================================
        #               Testing
        # ========================================
        print('Testing...')
        mrr_bert, map_bert, ndcg_bert, mrr_bert_list, map_bert_list, ndcg_bert_list = utils.testing(MODE, model,
                                                                                                    test_dataloader,
                                                                                                    device, test_index,
                                                                                                    bm25_top_n,
                                                                                                    mrr_bert_list,
                                                                                                    map_bert_list,
                                                                                                    ndcg_bert_list,
                                                                                                    mrr_bert, map_bert,
                                                                                                    ndcg_bert, rel_fed,
                                                                                                    fold_number,
                                                                                                    MAP_CUT, NDCG_CUT)
    print("  BM25 MRR:  " + "{:.4f}".format(mrr_bm25 / 5))
    print("  BM25 MAP:  " + "{:.4f}".format(map_bm25 / 5))
    print("  BM25 NDCG: " + "{:.4f}".format(ndcg_bm25 / 5))

    print("  BERT MRR:  " + "{:.4f}".format(mrr_bert / 5))
    print("  BERT MAP:  " + "{:.4f}".format(map_bert / 5))
    print("  BERT NDCG: " + "{:.4f}".format(ndcg_bert / 5))

    utils.t_test(mrr_bm25_list, mrr_bert_list, 'MRR')
    utils.t_test(map_bm25_list, map_bert_list, 'MAP')
    utils.t_test(ndcg_bm25_list, ndcg_bert_list, 'NDCG')
   
    stop = timeit.default_timer()
    wall_time = (stop - start) / 60 

    print('Time: ', wall_time, ' min')

    # utils.results_to_csv('./mrr_bm25_list.csv', mrr_bm25_list)
    # utils.results_to_csv('./mrr_bert_list.csv', mrr_bert_list)
    # utils.results_to_csv('./map_bm25_list.csv', map_bm25_list)
    # utils.results_to_csv('./map_bert_list.csv', map_bert_list)
    # utils.results_to_csv('./ndcg_bm25_list.csv', ndcg_bm25_list)
    # utils.results_to_csv('./ndcg_bert_list.csv', ndcg_bert_list)


# Testing part

In [7]:
EPOCHS = 2 
train_test()

# ========================================
#               Hyper-Parameters
Re-ranker
bert-base-uncased
2e-05
128
32
2
# ========================================
#               Experiment-Settings
CONVERSION TABLE:
[[0, 1, 2, 3, 4], [0, 1, 1, 1, 1]]
# ========================================
GPU Type: Tesla K80


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


======== Fold 1 / 5 ========
MRR:  0.7837
MAP:  0.3493
NDCG: 0.5011
45


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

======== Epoch 1 / 2 ========
Training...
  Batch   100  of    563.
  Batch   200  of    563.
  Batch   300  of    563.


KeyboardInterrupt: ignored

In [ ]:
LEARNING_RATE = 3e-5
EPOCHS = 1
train_test()

In [ ]:
LEARNING_RATE = 3e-5
EPOCHS = 2
train_test()

In [ ]:
CONVERSION_TABLE = [[0, 1, 2, 3, 4],[-0.5, 1, 0.8, 0.6, 0.3]] 

In [ ]:
LEARNING_RATE = 2e-5
EPOCHS = 2
train_test()

In [ ]:
LEARNING_RATE = 3e-5
EPOCHS = 1
train_test()

In [ ]:
LEARNING_RATE = 3e-5
EPOCHS = 2
train_test()

In [ ]:
CONVERSION_TABLE = [[0, 1, 2, 3, 4],[-1, 1, 0.7, 0.53, 0.4]] 

In [ ]:
LEARNING_RATE = 2e-5
EPOCHS = 2
train_test()

In [ ]:
LEARNING_RATE = 3e-5
EPOCHS = 1
train_test()

In [ ]:

LEARNING_RATE = 3e-5
EPOCHS = 2
train_test()